In [26]:
import os

In [27]:
%pwd

'd:\\text_summ_krish_naik\\Text-Summarizer-Project\\research'

In [28]:
os.chdir("../")

In [29]:
%pwd

'd:\\text_summ_krish_naik\\Text-Summarizer-Project'

In [30]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL: str
    local_data_file: Path 
    unzip_dir: Path



In [31]:
!pip install pyyaml


You should consider upgrading via the 'D:\text_summ_krish_naik\Text-Summarizer-Project\textS\Scripts\python.exe -m pip install --upgrade pip' command.


In [35]:
from src.textSummarizer.constants import * 
from src.textSummarizer.utils.common import read_yaml

In [39]:
from pathlib import Path
from src.textSummarizer.utils.common import read_yaml, create_directories
from src.textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.textSummarizer.entity.config_entity import DataIngestionConfig


class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(Path(config_filepath))  # ✅ Convert to Path
        self.params = read_yaml(Path(params_filepath))  # ✅ Convert to Path

        create_directories([self.config["artifacts_root"]])  # Ensure directory exists

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config["data_ingestion"]

        create_directories([config["root_dir"]])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config["root_dir"],
            source_URL=config["source_URL"],
            local_data_file=config["local_data_file"],
            unzip_dir=config["unzip_dir"],
        )

        return data_ingestion_config


In [40]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [41]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [42]:
import os

config_path = "config/config.yaml"
print("Config file exists:", os.path.exists(config_path))


Config file exists: True


In [45]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    # ✅ SKIP DOWNLOAD (since you manually downloaded)
    # data_ingestion.download_file()  # REMOVE or COMMENT this line

    # ✅ Extract the ZIP file
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2025-03-19 22:41:50,532: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-19 22:41:50,535: INFO: common: yaml file: config\params.yaml loaded successfully]
[2025-03-19 22:41:50,537: INFO: common: created directory at: artifacts]
[2025-03-19 22:41:50,538: INFO: common: created directory at: artifacts/data_ingestion]
